In [1]:
#include <stdlib.h>
#include <iostream>
#include "infoapi.hpp"

struct Ticket: public Record{
    string name;
    int    number;
    void on_save(ostream&out){ out << number << endl << name << endl; }
    void on_load(istream&in) { in >> number >> name; }
};

enum OP { GET_TICKET, PRINT_TICKETS };

class TicketsDatabase: public DatabaseWorker{
public:
    TicketsDatabase(unsigned pid, EventLog&log):DatabaseWorker(pid,log){}
private:
    void on_run(int tag)override{
        switch(tag)
        {
            case OP::GET_TICKET:
            {
                Ticket my_ticket;
IO_SECTION_BEGIN
                cout << "Please, print your name:" << endl; 
                cin >> my_ticket.name;
                my_ticket.number = rand()/25 + 1;
IO_SECTION_END
                
IO_SYNC_POINT(my_ticket)
                
                if(tickets.find(my_ticket.name)!=tickets.end()){
IO_SECTION_BEGIN
                    cout << "Your already took the ticket.";
IO_SECTION_END
                    return;
                }
                tickets[my_ticket.name] = my_ticket.number;
IO_SECTION_BEGIN                
                cout << "Your ticket number is:" << my_ticket.number;
IO_SECTION_END                
                break;
            }
            case OP::PRINT_TICKETS:
            {
IO_SECTION_BEGIN
                for(auto& t: tickets){
                    cout << "         Name:" << t.first  << endl
                         << "Ticket number:" << t.second << endl << endl;
                }
IO_SECTION_END
            }
        }
    }
    map<string,int> tickets;
};

EventLog event_log;

In [2]:
{
    TicketsDatabase tickets(0,event_log);
    tickets.run(OP::GET_TICKET);
}

In [3]:
{
    TicketsDatabase tickets(1,event_log);
    tickets.run(OP::PRINT_TICKETS);
}